# Archive statistics
## Completeness

In [ ]:
import ipywidgets as widgets
widgets.Widget.widget_types
import requests
import geopy
import pandas as pd
from IPython.display import display
%matplotlib inline

import seaborn as sns
import plotly
from ipywidgets import interact

import folium

from geopy.geocoders import Nominatim
geolocator = Nominatim()

from IPython.display import clear_output

import json

In [ ]:
from __future__ import print_function

from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

from traitlets import link

In [ ]:
aoi_picker = widgets.Text(value='Canberra',placeholder='update this field',
    description='placename search',
    disabled=False)
def handle_submit(sender):
    global location
    global center
    location = geolocator.geocode(aoi_picker.value)
    center = [location.latitude, location.longitude]
    print(center)
aoi_picker.on_submit(handle_submit)

In [ ]:
from_date_picker = widgets.Text(value='2000-12-31',placeholder='update this field',
    description='start date',
    disabled=False)
def handle_from_date(sender):
    print(from_date_picker.value)
from_date_picker.on_submit(handle_from_date)

In [ ]:
to_date_picker = widgets.Text(value='2001-01-01',placeholder='update this field',
    description='end date',
    disabled=False)
def handle_to_date(sender):
    print(to_date_picker.value)
to_date_picker.on_submit(handle_to_date)

In [ ]:
product_picker = widgets.SelectMultiple(
    options=['nbar_albers','nbart_albers', 'pq_albers','fc', 'water'],
    description='product:',
    disabled=False
)

In [ ]:
sensor_picker =widgets.SelectMultiple(
    options=['ls5','ls7', 'ls8_oli','ls8'],
    description='sensor:',
    disabled=False
)

In [ ]:
accordion = widgets.Accordion(children=[product_picker, sensor_picker, from_date_picker, to_date_picker, aoi_picker])
accordion.set_title(0, 'product')
accordion.set_title(1, 'sensor')
accordion.set_title(2, 'from date')
accordion.set_title(3, 'to date')
accordion.set_title(4, 'placename')
accordion

In [ ]:
# initialise the DrawControl to capture the geometry
update_map = widgets.Button(description="Update map on click")
def update_map_clicked(button):
    
    global dc
    dc = DrawControl(polygon={'shapeOptions': {'color': '#0000FF'}}, polyline={'shapeOptions': {'color': '#00FF00'}},
                  circle={'shapeOptions': {'color': '#0000FF'}}, rectangle={'shapeOptions': {'color': '#0000FF'}})
    # add the DrawControl to the map
    global m
    m = Map(center=center, zoom=zoom)
    m.add_control(dc)
    display(m)
display(update_map)
update_map.on_click(update_map_clicked)

In [ ]:
button = widgets.Button(description="Generate Report")
def on_button_clicked(button):
    print('Generating your report now, hold onto your shorts')

    print(product_picker.value, sensor_picker.value, from_date_picker.value, to_date_picker.value, dc.last_draw['geometry'])

display(button)
button.on_click(on_button_clicked)

In [ ]:
list = ['P0', 'P1', 'P2', 'P3', 'P4']
children = [accordion,update_map]
print(children)
tab = widgets.Tab(children=children)
tab

In [ ]:
ul_lon, ul_lat = 77.83, 17.84 # upper left longitude, latitude
lr_lon, lr_lat = 78.00, 17.67 # lower right longitude, latitude
date_start, date_end = from_date_picker.value, to_date_picker.value  # start date and end date for time range selection  
sensor_list=["olitirs8", "tm5", "etm7"] # list of sensors to use

polygon_list = [[ul_lon, ul_lat], [lr_lon, ul_lat],[lr_lon, lr_lat],[ul_lon, lr_lat],[ul_lon, ul_lat]]

wrs_query = 'http://api.remotepixel.ca/wrs2tiles?search=poly:'+str(polygon_list)

In [ ]:
post_query = requests.get(wrs_query)
wrs_search_result = json.loads(post_query.text)

path_row = []
for item in wrs_search_result['results']:
    path_row.append(str(item['path'])+"_"+str(item['row']))
path_row

In [ ]:
tips = sns.load_dataset('tips')

In [ ]:
@interact(hue=['smoker','sex', 'time','day'])
def plot(hue):
    _ = sns.pairplot(tips, hue=hue)